## הטמעות

בדוגמה הקודמת שלנו, עבדנו עם וקטורים של שקי מילים בממדים גבוהים באורך `vocab_size`, והמרנו באופן מפורש מווקטורי ייצוג מיקום בממדים נמוכים לייצוג דל של one-hot. ייצוג ה-one-hot הזה אינו יעיל מבחינת זיכרון, בנוסף, כל מילה מטופלת באופן עצמאי, כלומר וקטורי one-hot מקודדים אינם מבטאים שום דמיון סמנטי בין מילים.

ביחידה זו, נמשיך לחקור את מאגר הנתונים **News AG**. כדי להתחיל, נטען את הנתונים ונקבל כמה הגדרות מהמחברת הקודמת.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## מהו הטמעה?

הרעיון של **הטמעה** הוא לייצג מילים באמצעות וקטורים צפופים בממדים נמוכים, שמשקפים בצורה כלשהי את המשמעות הסמנטית של המילה. בהמשך נדון כיצד לבנות הטמעות מילים משמעותיות, אבל כרגע נחשוב על הטמעות כדרך להקטין את הממדיות של וקטור המילה.

לכן, שכבת הטמעה תקבל מילה כקלט ותפיק וקטור פלט בגודל `embedding_size` שנקבע מראש. במובן מסוים, זה מאוד דומה לשכבת `Linear`, אבל במקום לקבל וקטור מקודד בשיטת one-hot, היא תוכל לקבל מספר מילה כקלט.

על ידי שימוש בשכבת הטמעה כשכבה הראשונה ברשת שלנו, נוכל לעבור ממודל bag-of-words למודל **embedding bag**, שבו קודם כל נמיר כל מילה בטקסט שלנו להטמעה המתאימה שלה, ואז נחשב פונקציית צבירה כלשהי על כל ההטמעות הללו, כמו `sum`, `average` או `max`.

![תמונה המציגה מסווג הטמעות עבור חמש מילים ברצף.](../../../../../translated_images/he/embedding-classifier-example.b77f021a7ee67eee.webp)

רשת העצבים המסווגת שלנו תתחיל עם שכבת הטמעה, לאחר מכן שכבת צבירה, ולבסוף מסווג ליניארי מעליה:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### התמודדות עם גודל משתנה של רצף משתנים

כתוצאה מהארכיטקטורה הזו, יש צורך ליצור מיניבאצ'ים לרשת שלנו בצורה מסוימת. ביחידה הקודמת, כאשר השתמשנו ב-bag-of-words, כל הטנזורים של BoW במיניבאץ' היו בגודל שווה `vocab_size`, ללא קשר לאורך האמיתי של רצף הטקסט שלנו. ברגע שעוברים לשימוש ב-word embeddings, נמצא את עצמנו עם מספר משתנה של מילים בכל דוגמת טקסט, וכאשר משלבים את הדוגמאות הללו למיניבאצ'ים, נצטרך להוסיף ריפוד.

ניתן לעשות זאת באמצעות אותה טכניקה של מתן פונקציית `collate_fn` למקור הנתונים:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### אימון מסווג embedding

עכשיו, לאחר שהגדרנו את ה-dataloader בצורה נכונה, אנחנו יכולים לאמן את המודל באמצעות פונקציית האימון שהגדרנו ביחידה הקודמת:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **הערה**: אנו מאמנים כאן רק עבור 25k רשומות (פחות מאפוק מלא אחד) מטעמי זמן, אך ניתן להמשיך באימון, לכתוב פונקציה לאימון עבור מספר אפוקים, ולנסות עם פרמטר קצב הלמידה כדי להגיע לדיוק גבוה יותר. אתם אמורים להיות מסוגלים להגיע לדיוק של כ-90%.


### שכבת EmbeddingBag וייצוג רצפים באורך משתנה

בארכיטקטורה הקודמת, היינו צריכים לרפד את כל הרצפים לאורך אחיד כדי להתאים אותם למיני-באטץ'. זו לא הדרך היעילה ביותר לייצג רצפים באורך משתנה - גישה אחרת תהיה להשתמש בוקטור **offset**, שמחזיק את ההיסטים של כל הרצפים המאוחסנים בוקטור גדול אחד.

![תמונה המציגה ייצוג רצף עם היסטים](../../../../../translated_images/he/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: בתמונה למעלה, אנו מציגים רצף של תווים, אך בדוגמה שלנו אנו עובדים עם רצפים של מילים. עם זאת, העיקרון הכללי של ייצוג רצפים באמצעות וקטור היסטים נשאר זהה.

כדי לעבוד עם ייצוג היסטים, אנו משתמשים בשכבת [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html). היא דומה ל-`Embedding`, אך היא מקבלת וקטור תוכן ווקטור היסטים כקלט, והיא כוללת גם שכבת ממוצע, שיכולה להיות `mean`, `sum` או `max`.

הנה רשת מעודכנת שמשתמשת ב-`EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

כדי להכין את מערך הנתונים לאימון, עלינו לספק פונקציית המרה שתכין את וקטור ההיסט:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

שימו לב, שבניגוד לכל הדוגמאות הקודמות, הרשת שלנו כעת מקבלת שני פרמטרים: וקטור נתונים ווקטור הזזה, שהם בגדלים שונים. באופן דומה, טוען הנתונים שלנו מספק לנו 3 ערכים במקום 2: גם וקטורי הטקסט וגם וקטורי ההזזה מסופקים כתכונות. לכן, עלינו להתאים מעט את פונקציית האימון שלנו כדי לטפל בכך:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## הטמעות סמנטיות: Word2Vec

בדוגמה הקודמת שלנו, שכבת ההטמעה של המודל למדה למפות מילים לייצוג וקטורי, אך לייצוג הזה לא הייתה משמעות סמנטית רבה. יהיה נחמד ללמוד ייצוג וקטורי כזה, שבו מילים דומות או מילים נרדפות יתאימו לוקטורים שקרובים זה לזה במונחים של מרחק וקטורי כלשהו (למשל, מרחק אוקלידי).

כדי לעשות זאת, עלינו לבצע אימון מוקדם למודל ההטמעה שלנו על אוסף טקסטים גדול בצורה מסוימת. אחת הדרכים הראשונות לאמן הטמעות סמנטיות נקראת [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). היא מבוססת על שתי ארכיטקטורות עיקריות המשמשות ליצירת ייצוג מבוזר של מילים:

- **Continuous bag-of-words** (CBoW) — בארכיטקטורה זו, אנו מאמנים את המודל לנבא מילה מתוך ההקשר שסביבה. בהינתן הנגרם $(W_{-2},W_{-1},W_0,W_1,W_2)$, מטרת המודל היא לנבא את $W_0$ מתוך $(W_{-2},W_{-1},W_1,W_2)$.
- **Continuous skip-gram** הוא ההפך מ-CBoW. המודל משתמש בחלון ההקשר של המילים שסביב כדי לנבא את המילה הנוכחית.

CBoW מהיר יותר, בעוד ש-Skip-Gram איטי יותר, אך מבצע עבודה טובה יותר בייצוג מילים נדירות.

![תמונה המציגה את האלגוריתמים CBoW ו-Skip-Gram להמרת מילים לוקטורים.](../../../../../translated_images/he/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

כדי להתנסות בהטמעת Word2Vec שאומנה מראש על מאגר הנתונים של Google News, נוכל להשתמש בספריית **gensim**. להלן נמצא את המילים שהכי דומות ל-'neural'

> **Note:** כשאתם יוצרים וקטורי מילים בפעם הראשונה, ההורדה שלהם עשויה לקחת זמן!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


אנחנו יכולים גם לחשב הטמעות וקטוריות מהמילה, לשימוש באימון מודל סיווג (אנחנו מציגים רק את 20 הרכיבים הראשונים של הווקטור לצורך בהירות):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

הדבר הנהדר בהטמעות סמנטיות הוא שניתן לשנות את קידוד הווקטור כדי לשנות את הסמנטיקה. לדוגמה, אנו יכולים לבקש למצוא מילה, שהייצוג הווקטורי שלה יהיה קרוב ככל האפשר למילים *מלך* ו-*אישה*, ורחוק ככל האפשר מהמילה *גבר*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

גם CBoW וגם Skip-Grams הם שיטות ליצירת הטמעות "חזויות", בכך שהן מתמקדות רק בהקשרים מקומיים. Word2Vec אינו מנצל הקשר גלובלי.

**FastText** מבוסס על Word2Vec, אך לומד ייצוגי וקטורים לכל מילה ול-n-grams של תווים שנמצאים בתוך כל מילה. הערכים של הייצוגים הללו ממוצעים לווקטור אחד בכל שלב של האימון. למרות שזה מוסיף חישובים נוספים משמעותיים בשלב הקדם-אימון, זה מאפשר להטמעות המילים לקודד מידע על תת-מילים.

שיטה נוספת, **GloVe**, מנצלת את הרעיון של מטריצת שכיחויות משותפות (co-occurrence matrix), ומשתמשת בשיטות נוירוניות כדי לפרק את מטריצת השכיחויות לוקטורי מילים יותר אקספרסיביים ולא ליניאריים.

אתם יכולים להתנסות בדוגמה על ידי שינוי ההטמעות ל-FastText ו-GloVe, מכיוון ש-gensim תומך בכמה מודלים שונים של הטמעת מילים.


## שימוש באמבדינגים מוכנים מראש ב-PyTorch

ניתן לשנות את הדוגמה שלמעלה כדי למלא מראש את המטריצה בשכבת האמבדינג שלנו עם אמבדינגים סמנטיים, כמו Word2Vec. יש לקחת בחשבון שהמילונים של האמבדינגים המוכנים מראש ושל קורפוס הטקסט שלנו כנראה לא יתאימו, ולכן נאתחל את המשקלים עבור המילים החסרות עם ערכים אקראיים:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


עכשיו בואו נאמן את המודל שלנו. שימו לב שזמן האימון של המודל ארוך משמעותית בהשוואה לדוגמה הקודמת, בשל גודל שכבת ההטמעה הגדול יותר, ולכן מספר הפרמטרים גבוה בהרבה. בנוסף, בגלל זה, ייתכן שנצטרך לאמן את המודל שלנו על יותר דוגמאות אם נרצה להימנע מהתאמת יתר.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

במקרה שלנו, איננו רואים עלייה משמעותית בדיוק, מה שסביר להניח נובע מאוצר מילים שונה למדי.  
כדי להתגבר על הבעיה של אוצר מילים שונה, ניתן להשתמש באחת מהפתרונות הבאים:  
* לאמן מחדש את מודל ה-word2vec על אוצר המילים שלנו  
* לטעון את מערך הנתונים שלנו עם אוצר המילים ממודל ה-word2vec שאומן מראש. ניתן לציין את אוצר המילים שישמש לטעינת מערך הנתונים במהלך הטעינה.  

הגישה השנייה נראית קלה יותר, במיוחד מכיוון שמסגרת `torchtext` של PyTorch מכילה תמיכה מובנית בהטמעות.  
לדוגמה, ניתן ליצור אוצר מילים מבוסס GloVe באופן הבא:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


אוצר המילים הטעון כולל את הפעולות הבסיסיות הבאות:  
* המילון `vocab.stoi` מאפשר לנו להמיר מילה למספר האינדקס שלה במילון  
* `vocab.itos` עושה את ההפך - ממיר מספר למילה  
* `vocab.vectors` הוא מערך של וקטורי ההטמעה, כך שכדי לקבל את ההטמעה של מילה `s` עלינו להשתמש ב-`vocab.vectors[vocab.stoi[s]]`  

הנה דוגמה למניפולציה על הטמעות כדי להדגים את המשוואה **kind-man+woman = queen** (הייתי צריך לכוונן קצת את המקדמים כדי שזה יעבוד):  


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

כדי לאמן את מסווג באמצעות ההטמעות הללו, תחילה עלינו לקודד את מערך הנתונים שלנו באמצעות אוצר המילים של GloVe:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

כפי שראינו לעיל, כל ההטמעות הווקטוריות מאוחסנות במטריצת `vocab.vectors`. זה הופך את הטעינה של המשקלים הללו למשקלים של שכבת ההטמעה לקלה במיוחד באמצעות העתקה פשוטה:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

עכשיו בואו נאמן את המודל שלנו ונראה אם נקבל תוצאות טובות יותר:


In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

אחת הסיבות לכך שאיננו רואים עלייה משמעותית בדיוק היא בשל העובדה שחלק מהמילים ממאגר הנתונים שלנו חסרות באוצר המילים של GloVe שהוכשר מראש, ולכן הן למעשה מתעלמות. כדי להתגבר על עובדה זו, אנו יכולים לאמן את ההטמעות שלנו על מאגר הנתונים שלנו.


## הטמעות בהקשר

אחת המגבלות המרכזיות של ייצוגי הטמעות מסורתיים שהוכנו מראש, כמו Word2Vec, היא בעיית הבהרת המשמעות של מילים. בעוד שהטמעות שהוכנו מראש יכולות ללכוד חלק מהמשמעות של מילים בהקשר, כל המשמעויות האפשריות של מילה מקודדות באותה הטמעה. זה יכול לגרום לבעיות במודלים בהמשך, מכיוון שלמילים רבות, כמו המילה 'play', יש משמעויות שונות בהתאם להקשר שבו הן משמשות.

לדוגמה, המילה 'play' בשני המשפטים הבאים יש לה משמעות שונה לחלוטין:
- הלכתי ל**הצגה** בתיאטרון.
- ג'ון רוצה **לשחק** עם חבריו.

הטמעות שהוכנו מראש מייצגות את שתי המשמעויות הללו של המילה 'play' באותה הטמעה. כדי להתגבר על מגבלה זו, עלינו לבנות הטמעות המבוססות על **מודל שפה**, שמאומן על מאגר טקסט גדול, ו*יודע* כיצד מילים יכולות להשתלב בהקשרים שונים. דיון בהטמעות בהקשר חורג מתחום המדריך הזה, אך נחזור אליהן כשנדבר על מודלי שפה ביחידה הבאה.



---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור הסמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
